# Bias Detection & Data Quality Analysis

**Purpose**: Identify data bias, missing patterns, outliers, and systematic data quality issues

**Date**: January 12, 2026

## Objectives
1. Detect temporal bias (missing years, incomplete periods)
2. Identify geographic bias (missing countries, regions)
3. Find systematic outliers and anomalies
4. Analyze data completeness patterns
5. Detect measurement bias and inconsistencies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Database connection
DB_CONFIG = {
    'host': '172.18.0.1',
    'port': 5432,
    'database': 'lianel_energy',
    'user': 'airflow',
    'password': 'P9xK2mN7vQ4wR8tY3sL6hJ5nB1cV0zX'
}

connection_string = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
engine = create_engine(connection_string)

print("✅ Database connection established")

## 1. Temporal Bias Detection

In [ ]:
# Load ML dataset
query = """
SELECT 
    cntr_code,
    year,
    total_energy_gwh,
    renewable_energy_gwh,
    fossil_energy_gwh,
    pct_renewable,
    yoy_change_total_energy_pct
FROM ml_dataset_forecasting_v1
ORDER BY cntr_code, year
"""

df = pd.read_sql(query, engine)

# Check temporal coverage
temporal_coverage = df.groupby('cntr_code').agg({
    'year': ['min', 'max', 'count', lambda x: sorted(x).tolist()]
}).round(0)

temporal_coverage.columns = ['min_year', 'max_year', 'record_count', 'years_list']

# Identify missing years for each country
all_years = sorted(df['year'].unique())
expected_years_per_country = len(all_years)

temporal_coverage['missing_years'] = temporal_coverage['years_list'].apply(
    lambda x: [y for y in all_years if y not in x]
)
temporal_coverage['missing_count'] = temporal_coverage['missing_years'].apply(len)
temporal_coverage['coverage_pct'] = (1 - temporal_coverage['missing_count'] / expected_years_per_country) * 100

print("📅 Temporal Coverage Analysis:")
print(f"Expected years per country: {expected_years_per_country} ({all_years[0]}-{all_years[-1]})")
print(f"\nCountries with incomplete temporal coverage:")
incomplete = temporal_coverage[temporal_coverage['missing_count'] > 0].sort_values('missing_count', ascending=False)
print(incomplete[['min_year', 'max_year', 'record_count', 'missing_count', 'coverage_pct']].to_string())

# Visualize temporal coverage
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Coverage percentage by country
ax1 = axes[0]
coverage_sorted = temporal_coverage.sort_values('coverage_pct')
ax1.barh(coverage_sorted.index, coverage_sorted['coverage_pct'], color='steelblue', alpha=0.7)
ax1.axvline(x=100, color='green', linestyle='--', linewidth=2, label='100% Coverage')
ax1.set_xlabel('Coverage Percentage')
ax1.set_ylabel('Country')
ax1.set_title('Temporal Coverage by Country')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='x')

# Plot 2: Missing years heatmap
ax2 = axes[1]
missing_matrix = []
for country in df['cntr_code'].unique():
    country_years = set(df[df['cntr_code'] == country]['year'])
    missing = [1 if year not in country_years else 0 for year in all_years]
    missing_matrix.append(missing)

missing_df = pd.DataFrame(missing_matrix, index=df['cntr_code'].unique(), columns=all_years)
sns.heatmap(missing_df, cmap='Reds', cbar_kws={'label': 'Missing (1=Missing, 0=Present)'}, 
            ax=ax2, linewidths=0.5)
ax2.set_xlabel('Year')
ax2.set_ylabel('Country')
ax2.set_title('Missing Data Heatmap (Red = Missing Year)')

plt.tight_layout()
plt.show()

## 2. Geographic Bias Detection

In [ ]:
# Check for missing countries
countries_query = """
SELECT DISTINCT country_code, country_name 
FROM dim_country 
ORDER BY country_code
"""
all_countries = pd.read_sql(countries_query, engine)

# Countries in ML dataset
ml_countries = set(df['cntr_code'].unique())
all_country_codes = set(all_countries['country_code'].unique())

missing_countries = all_country_codes - ml_countries
extra_countries = ml_countries - all_country_codes

print("🌍 Geographic Coverage Analysis:")
print(f"Total countries in dim_country: {len(all_country_codes)}")
print(f"Countries in ML dataset: {len(ml_countries)}")
print(f"Missing countries: {len(missing_countries)}")
if missing_countries:
    missing_names = all_countries[all_countries['country_code'].isin(missing_countries)][['country_code', 'country_name']]
    print("\nMissing countries:")
    print(missing_names.to_string(index=False))

# Analyze data volume by country
country_stats = df.groupby('cntr_code').agg({
    'total_energy_gwh': ['mean', 'sum', 'std', 'count'],
    'pct_renewable': 'mean'
}).round(2)
country_stats.columns = ['avg_energy', 'total_energy', 'std_energy', 'record_count', 'avg_renewable_pct']

# Identify countries with very low or very high energy consumption
country_stats['energy_rank'] = country_stats['avg_energy'].rank(ascending=False)
country_stats = country_stats.sort_values('avg_energy', ascending=False)

print("\n📊 Country Statistics (sorted by average energy):")
print(country_stats.head(10).to_string())

# Visualize geographic distribution
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Average energy by country
ax1 = axes[0, 0]
top_countries = country_stats.head(15)
ax1.barh(top_countries.index, top_countries['avg_energy'], color='steelblue', alpha=0.7)
ax1.set_xlabel('Average Energy (GWh)')
ax1.set_ylabel('Country')
ax1.set_title('Top 15 Countries by Average Energy Consumption')
ax1.grid(True, alpha=0.3, axis='x')

# Plot 2: Average renewable percentage by country
ax2 = axes[0, 1]
renewable_sorted = country_stats.sort_values('avg_renewable_pct', ascending=False).head(15)
ax2.barh(renewable_sorted.index, renewable_sorted['avg_renewable_pct'], color='green', alpha=0.7)
ax2.set_xlabel('Average Renewable Percentage')
ax2.set_ylabel('Country')
ax2.set_title('Top 15 Countries by Renewable Energy Percentage')
ax2.grid(True, alpha=0.3, axis='x')

# Plot 3: Energy distribution by country
ax3 = axes[1, 0]
country_stats['avg_energy'].hist(bins=20, ax=ax3, color='orange', alpha=0.7, edgecolor='black')
ax3.set_xlabel('Average Energy (GWh)')
ax3.set_ylabel('Frequency')
ax3.set_title('Distribution of Average Energy by Country')
ax3.grid(True, alpha=0.3)

# Plot 4: Record count by country
ax4 = axes[1, 1]
ax4.barh(country_stats.index, country_stats['record_count'], color='purple', alpha=0.7)
ax4.set_xlabel('Number of Records')
ax4.set_ylabel('Country')
ax4.set_title('Data Completeness by Country (Record Count)')
ax4.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## 3. Outlier Detection

In [ ]:
# Detect outliers using IQR method
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Detect outliers in key metrics
outliers_total = detect_outliers_iqr(df, 'total_energy_gwh')
outliers_renewable = detect_outliers_iqr(df, 'pct_renewable')
outliers_yoy = detect_outliers_iqr(df[df['yoy_change_total_energy_pct'].abs() < 100], 'yoy_change_total_energy_pct')

print("🔍 Outlier Detection Results:")
print(f"\nTotal Energy Outliers: {len(outliers_total[0])} records")
print(f"  Lower bound: {outliers_total[1]:.2f} GWh")
print(f"  Upper bound: {outliers_total[2]:.2f} GWh")
if len(outliers_total[0]) > 0:
    print("\nTop outliers:")
    print(outliers_total[0][['cntr_code', 'year', 'total_energy_gwh']].head(10).to_string(index=False))

print(f"\nRenewable Percentage Outliers: {len(outliers_renewable[0])} records")
print(f"  Lower bound: {outliers_renewable[1]:.2f}%")
print(f"  Upper bound: {outliers_renewable[2]:.2f}%")
if len(outliers_renewable[0]) > 0:
    print("\nTop outliers:")
    print(outliers_renewable[0][['cntr_code', 'year', 'pct_renewable']].head(10).to_string(index=False))

print(f"\nYoY Change Outliers (excluding extremes): {len(outliers_yoy[0])} records")
if len(outliers_yoy[0]) > 0:
    print("\nTop outliers:")
    print(outliers_yoy[0][['cntr_code', 'year', 'yoy_change_total_energy_pct']].head(10).to_string(index=False))

# Visualize outliers
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Box plot for total energy
ax1 = axes[0, 0]
df.boxplot(column='total_energy_gwh', ax=ax1, vert=False)
ax1.set_xlabel('Total Energy (GWh)')
ax1.set_title('Total Energy Distribution (Box Plot)')
ax1.grid(True, alpha=0.3)

# Plot 2: Box plot for renewable percentage
ax2 = axes[0, 1]
df.boxplot(column='pct_renewable', ax=ax2, vert=False)
ax2.set_xlabel('Renewable Percentage')
ax2.set_title('Renewable Percentage Distribution (Box Plot)')
ax2.grid(True, alpha=0.3)

# Plot 3: Scatter plot with outliers highlighted
ax3 = axes[1, 0]
normal_data = df[~df.index.isin(outliers_total[0].index)]
ax3.scatter(normal_data['year'], normal_data['total_energy_gwh'], 
           alpha=0.5, label='Normal', s=20)
if len(outliers_total[0]) > 0:
    ax3.scatter(outliers_total[0]['year'], outliers_total[0]['total_energy_gwh'], 
               color='red', label='Outliers', s=50, marker='x')
ax3.set_xlabel('Year')
ax3.set_ylabel('Total Energy (GWh)')
ax3.set_title('Total Energy Over Time (Outliers Highlighted)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Z-score distribution
ax4 = axes[1, 1]
from scipy import stats
z_scores = np.abs(stats.zscore(df['total_energy_gwh'].dropna()))
ax4.hist(z_scores, bins=50, alpha=0.7, edgecolor='black', color='purple')
ax4.axvline(x=3, color='red', linestyle='--', linewidth=2, label='Z=3 (Outlier threshold)')
ax4.set_xlabel('Z-Score (Absolute)')
ax4.set_ylabel('Frequency')
ax4.set_title('Z-Score Distribution for Total Energy')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Data Completeness Analysis

In [ ]:
# Check for missing values
missing_analysis = df.isnull().sum()
missing_pct = (missing_analysis / len(df)) * 100

print("📋 Missing Value Analysis:")
missing_df = pd.DataFrame({
    'Missing Count': missing_analysis,
    'Missing Percentage': missing_pct
}).sort_values('Missing Count', ascending=False)
print(missing_df[missing_df['Missing Count'] > 0].to_string())

# Check for zero values (potential data quality issues)
zero_analysis = {}
for col in ['total_energy_gwh', 'renewable_energy_gwh', 'fossil_energy_gwh']:
    zero_count = (df[col] == 0).sum()
    zero_analysis[col] = {
        'zero_count': zero_count,
        'zero_pct': (zero_count / len(df)) * 100
    }

print("\n⚠️ Zero Value Analysis:")
for col, stats in zero_analysis.items():
    print(f"  {col}: {stats['zero_count']} zeros ({stats['zero_pct']:.2f}%)")

# Check for invalid percentages
invalid_pct = df[
    (df['pct_renewable'] < 0) | 
    (df['pct_renewable'] > 100) |
    (df['pct_renewable'].isnull())
]
print(f"\n❌ Invalid Renewable Percentages: {len(invalid_pct)} records")
if len(invalid_pct) > 0:
    print(invalid_pct[['cntr_code', 'year', 'pct_renewable', 'total_energy_gwh']].to_string(index=False))

# Check for data completeness by year
completeness_by_year = df.groupby('year').agg({
    'cntr_code': 'count',
    'total_energy_gwh': lambda x: (x > 0).sum(),
    'renewable_energy_gwh': lambda x: (x > 0).sum(),
    'fossil_energy_gwh': lambda x: (x > 0).sum()
})
completeness_by_year.columns = ['total_records', 'has_total_energy', 'has_renewable', 'has_fossil']
completeness_by_year['fossil_completeness_pct'] = (completeness_by_year['has_fossil'] / completeness_by_year['total_records']) * 100

print("\n📊 Data Completeness by Year:")
print(completeness_by_year.to_string())

# Visualize completeness
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Completeness by year
ax1 = axes[0]
x = completeness_by_year.index
width = 0.25
ax1.bar(x - width, completeness_by_year['has_total_energy'], width, label='Total Energy', alpha=0.7)
ax1.bar(x, completeness_by_year['has_renewable'], width, label='Renewable', alpha=0.7)
ax1.bar(x + width, completeness_by_year['has_fossil'], width, label='Fossil', alpha=0.7)
ax1.set_xlabel('Year')
ax1.set_ylabel('Number of Records')
ax1.set_title('Data Completeness by Year')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# Plot 2: Fossil completeness percentage
ax2 = axes[1]
ax2.plot(completeness_by_year.index, completeness_by_year['fossil_completeness_pct'], 
         marker='o', linewidth=2, markersize=8, color='red')
ax2.axhline(y=100, color='green', linestyle='--', linewidth=2, label='100% Complete')
ax2.set_xlabel('Year')
ax2.set_ylabel('Fossil Data Completeness (%)')
ax2.set_title('Fossil Energy Data Completeness Over Time')
ax2.set_ylim([0, 105])
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Key Findings:")
print(f"  - Years with incomplete fossil data: {completeness_by_year[completeness_by_year['fossil_completeness_pct'] < 100].index.tolist()}")
print(f"  - Missing values: {missing_analysis.sum()} total missing values across all columns")

## 5. Summary & Recommendations

### Key Bias Issues Identified

1. **Temporal Bias**: Missing years for some countries
2. **Data Completeness Bias**: 2016-2017 missing fossil data
3. **Geographic Bias**: Potential missing countries
4. **Outlier Bias**: Extreme values that may skew analysis

### Recommendations

1. **Flag incomplete data**: Add data quality flags to ML datasets
2. **Handle outliers**: Decide on outlier treatment (remove, cap, or investigate)
3. **Fill missing years**: Investigate why some countries have missing years
4. **Re-ingest incomplete periods**: Re-run ingestion for 2016-2017
5. **Document data limitations**: Create data quality documentation